In [26]:
import pandas as pd
import numpy as np
from siuba import *
from calitp_data_analysis.sql import *
from calitp_data_analysis.tables import tbls

In [2]:
pd.set_option("display.max_columns", 100)

In [3]:
bc = to_snakecase(
    pd.read_excel(
        "gs://calitp-analytics-data/data-analyses/grants/Grant+Projects_3_17_2022.xlsx"
    )
)

In [4]:
bc.funding_program.value_counts()

5310 Trad          964
Section 5311       590
CARES Act          166
CRRSAA             153
5310 Exp           151
Section 5311(f)    109
5339 (State)        95
Toll Credits        52
ARPA                50
5339 (National)     48
CARES Act (F)       43
5311(f) Cont        41
CMAQ (FTA 5311)     31
5311(f) Round 2     27
Name: funding_program, dtype: int64

In [5]:
df = bc>>filter(_.funding_program.str.contains('5310'))


In [6]:
len(df>>count(_.organization_name))

147

In [7]:
df.sample()

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,project_closed_by,project_closed_date,project_closed_time
269,2017,5310 Trad,CA-2017-169 | 0017000186-T,2017,City of Fontana,BCG0000977,Purchase Replacement < 30 Ft Bus,111204,64AC18-00697,73000.0,73000.0,0.0,73000.0,0,Open,NaN,NaN,NaN


In [8]:
df.project_year.value_counts()

2019    604
2017    507
2018      4
Name: project_year, dtype: int64

In [9]:
(
    df
    >> filter(_.organization_name.str.contains("Dignity"))
    >> group_by(_.grant_number, _.funding_program)
    >> count(_.organization_name)
)

,grant_number,funding_program,organization_name,n
0,CA-2017-169 | 0017000186-E,5310 Exp,Dignity Community Care,1
1,CA-2017-169 | 0017000186-E,5310 Exp,Dignity Health Connected Living,1
2,CA-2017-169 | 0017000186-T,5310 Trad,Dignity Community Care,10
3,CA-2017-169 | 0017000186-T,5310 Trad,Dignity Health Connected Living,6
4,CA-2020-244 | 0020000273-T,5310 Trad,Dignity Health Connected Living,4


In [10]:
##same grantnumber, different names

In [11]:
df>>group_by(_.grant_number)>>summarize(unique_orgs = _.organization_name.nunique())

,grant_number,unique_orgs
0,CA-2017-169 | 0017000186-E,57
1,CA-2017-169 | 0017000186-T,93
2,CA-2020-244 | 0020000273-E,50
3,CA-2020-244 | 0020000273-T,74


In [12]:
df>>group_by(_.organization_name)>>count(_.grant_number)

,organization_name,grant_number,n
0,ARC Bakersfield,CA-2017-169 | 0017000186-T,20
1,ARC Imperial Valley,CA-2017-169 | 0017000186-E,2
2,ARC Imperial Valley,CA-2020-244 | 0020000273-E,2
3,ARC Imperial Valley,CA-2020-244 | 0020000273-T,4
4,Able Industries,CA-2017-169 | 0017000186-T,6
...,...,...,...
269,Wilshire Community Services,CA-2017-169 | 0017000186-E,1
270,Wilshire Community Services,CA-2020-244 | 0020000273-E,1
271,Work Training Center,CA-2017-169 | 0017000186-T,7
272,Yolo County Transportation District,CA-2017-169 | 0017000186-E,2


In [13]:
from shared_utils import geography_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [14]:
org = geography_utils.aggregate_by_geography(
    df,
    group_cols=["organization_name"],
    nunique_cols=["grant_fiscal_year", "grant_number", "contract_number"],
    count_cols=["upin"],
    sum_cols=[
        "allocationamount",
        "encumbered_amount",
        "expendedamount",
        "activebalance",
        "closedoutbalance",
    ],
    mean_cols=[
        "allocationamount",
        "encumbered_amount",
        "expendedamount",
        "activebalance",
        "closedoutbalance",
    ],
    rename_cols=True,
)

In [15]:
org

,organization_name,activebalance_sum,allocationamount_sum,closedoutbalance_sum,encumbered_amount_sum,expendedamount_sum,activebalance_mean,allocationamount_mean,closedoutbalance_mean,encumbered_amount_mean,expendedamount_mean,upin_count,contract_number_nunique,grant_fiscal_year_nunique,grant_number_nunique
0,Amador Transit,-268853.00,469995.0,0,738848.0,738848.00,-67213.25000,117498.750,0,184712.000,184712.00000,4,3,2,3
1,"Angel View, Inc.",25729.61,554995.0,0,774990.0,529265.39,3216.20125,69374.375,0,96873.750,66158.17375,8,4,2,4
2,ARC Imperial Valley,-44628.20,549178.0,0,865145.0,593806.20,-5578.52500,68647.250,0,108143.125,74225.77500,8,3,2,3
3,Area 1 Agency on Aging,0.00,146534.0,0,146534.0,146534.00,0.00000,146534.000,0,146534.000,146534.00000,1,1,1,1
4,"Asian Community Center of Sacramento Valley, I...",541809.50,1425169.0,0,2320371.0,883359.50,27090.47500,71258.450,0,116018.550,44167.97500,20,5,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Self-Help for the Elderly,242098.00,242098.0,0,484196.0,0.00,30262.25000,30262.250,0,60524.500,0.00000,8,1,1,1
143,The Respite Inn,56000.00,56000.0,0,112000.0,0.00,28000.00000,28000.000,0,56000.000,0.00000,2,1,1,1
144,"Ukiah Senior Center, Inc.",56000.00,56000.0,0,112000.0,0.00,28000.00000,28000.000,0,56000.000,0.00000,2,1,1,1
145,United Cerebral Palsy Association Inc of Stani...,-239007.28,284800.0,0,569600.0,523807.28,-29875.91000,35600.000,0,71200.000,65475.91000,8,1,1,1


In [16]:
org["organization_name"] = (
    org["organization_name"]
    .str.replace(",", "")
    .str.replace(".", "")
    .str.replace("/", " ")
    #.str.replace("\\", "")
)

/tmp/ipykernel_5535/2494736175.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  org["organization_name"]


In [17]:
org_name= org>>select(_.organization_name)

In [18]:
org_name

,organization_name
0,Amador Transit
1,Angel View Inc
2,ARC Imperial Valley
3,Area 1 Agency on Aging
4,Asian Community Center of Sacramento Valley In...
...,...
142,Self-Help for the Elderly
143,The Respite Inn
144,Ukiah Senior Center Inc
145,United Cerebral Palsy Association Inc of Stani...


In [19]:
#pd.set_option("display.max_colwidth", 100)

In [20]:
org_name>>filter(_.organization_name.str.contains('/'))

,organization_name


In [21]:
org_name>>filter(_.organization_name.str.contains('Community Bridges'))

,organization_name
12,Community Bridges Liftline


## Read in Airtable Organization Information

In [33]:
itp= (tbls.mart_transit_database.dim_organizations()
 >>filter(_._is_current == True)
 >>collect())

In [34]:
itp.sample()

,key,source_record_id,name,organization_type,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,alias,is_public_entity,ntd_id,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to
7,3f5ec46caf318d18facde3706c7f72e7,rechioYJfbSnMyqP6,HBSS,Company,[],NaN,Offices in MA and MS,None,https://qryde.com/,None,None,Static OK,RT Incomplete,False,Unknown,[],False,None,False,False,True,2023-05-31 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00


In [35]:
itp = itp >> select(
    _.name,
    _.organization_type,
    _.roles,
    _.ntp_id,
    _.itp_id,
    _.alias,
    _.opm_id_drmt,
    _.mobility_services_operated,
    _.mpo_rtpa,
)

KeyError: "['ntp_id', 'opm_id_drmt', 'mobility_services_operated', 'mpo_rtpa'] not in index"

In [ ]:
itp['name'] = itp['name'].str.replace(',','').str.replace('.','').str.replace('/',' ')
#.str.replace("\\", "")

In [36]:
itp>>select(_.name)

,name
0,DAV
1,IBI
2,SAP
3,UTA
4,Aina
...,...
910,Berkeley Gateway Transportation Management Ass...
911,"Golden Gate Bridge, Highway and Transportation..."
912,Los Angeles County Metropolitan Transportation...
913,San Francisco Bay Area Water Emergency Transit...


In [37]:
itp.rename(columns = {'ntp_id':'ntd_id'}, inplace=True)

## Read in NTD Agency Info

In [38]:
ntd = to_snakecase(pd.read_excel("gs://calitp-analytics-data/data-analyses/5311 /2020_Agency_Information.xlsx"))

In [39]:
ntd = ntd>>filter(_.state=='CA')

In [40]:
ntd = ntd >> select(
    _.ntd_id,
    _.agency_name,
    _.doing_business_as,
    _.reporter_acronym,
    _.doing_business_as,
    _.organization_type,
    _.reporter_type,
    _.reported_by_ntd_id,
    _.city,
)

In [41]:
ntd

,ntd_id,agency_name,doing_business_as,reporter_acronym,organization_type,reporter_type,reported_by_ntd_id,city
884,90003,San Francisco Bay Area Rapid Transit District,NaN,BART,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Oakland
885,90004,Golden Empire Transit District,NaN,GET,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Bakersfield
887,90006,Santa Cruz Metropolitan Transit District,NaN,SCMTD,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Santa Cruz
888,90007,City of Modesto,Modesto Area Express,MAX,"City, County or Local Government Unit or Depar...",Full Reporter,NaN,Modesto
889,90008,City of Santa Monica,Big Blue Bus,NaN,"City, County or Local Government Unit or Depar...",Full Reporter,NaN,Santa Monica
...,...,...,...,...,...,...,...,...
2869,A0008,Kern Regional Transit,NaN,KT,"City, County or Local Government Unit or Depar...",Group Plan Sponsor,NaN,Bakersfield
2870,A0009,Humboldt Transit Authority,NaN,HTA,Independent Public Agency or Authority of Tran...,Group Plan Sponsor,NaN,Eureka
2952,A0013,Metropolitan Transportation Commission,NaN,MTC,"MPO, COG or Other Planning Agency",Group Plan Sponsor,NaN,San Francisco
2955,A0016,Redding Area Bus Authority,NaN,RABA,Independent Public Agency or Authority of Tran...,Group Plan Sponsor,NaN,Redding


In [42]:
ntd>>filter(_.agency_name.str.contains("Calaveras"))

,ntd_id,agency_name,doing_business_as,reporter_acronym,organization_type,reporter_type,reported_by_ntd_id,city
2829,9R02-99442,Calaveras Transit Agency,NaN,NaN,Independent Public Agency or Authority of Tran...,Rural Reporter,9R02,San Andreas


In [43]:
ntd['agency_name'] = ntd['agency_name'].str.replace(',','').str.replace('.','').str.replace('/',' ')
#.str.replace("\\", "")

/tmp/ipykernel_5535/1640863875.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ntd['agency_name'] = ntd['agency_name'].str.replace(',','').str.replace('.','').str.replace('/',' ')


## Merge NTD & ITP info

#### On Name

In [44]:
m = pd.merge(ntd, itp, left_on="agency_name", right_on="name", how="outer", indicator=True)

In [45]:
m._merge.value_counts()

right_only    733
both          185
left_only      56
Name: _merge, dtype: int64

In [46]:
m.sample(5)

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,alias,is_public_entity,ntd_id_y,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge
902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2b8220f64403d246a4c59d3c1b65daa9,reclYwhDwg5g7cPRY,Michigan Technological University Center for T...,University - Public,[],NaN,None,None,http://ctt.mtu.edu/,None,None,Static OK,RT Incomplete,False,Unknown,[],False,None,False,False,True,2023-05-27 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,right_only
959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5412f584eaf069e28b121d1b6f45f339,rec7MJohfRfqjDr6I,Butte-Glenn Community College District,University - Public,[],NaN,None,03 - Marysville,www.butte.edu,Other Public Transit,8752674455,Static Incomplete,RT Incomplete,True,No,[],False,None,False,False,True,2023-04-29 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,right_only
265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,680f6c4122f863f34953ac015d668a3d,recgUyVPkimiIL1yV,PlanIt,Company,[],NaN,Swedish company serving the Nordics.\n\nPLANit...,None,https://www.planit.se/en/,None,None,Static OK,RT Incomplete,False,Unknown,[],False,None,False,False,True,2023-04-29 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,right_only
766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102b5e5027dcd2280a37df68d1d14744,recAmb8vIfIzIKn87,"Veolia Transit Services, Inc",Company,[],NaN,None,None,https://www.veolianorthamerica.com/,None,None,Static OK,RT Incomplete,False,Unknown,[],False,None,False,False,True,2023-05-25 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,right_only
483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6cec4610303196d70751357c2becd681,recKzKiI3PAoTWvhi,City of Reedley,City/Town,[],NaN,None,06 - Fresno,https://reedley.ca.gov/,None,None,Static OK,RT Incomplete,False,Unknown,[],True,None,False,False,True,2023-05-25 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,right_only


In [47]:
nomatch1 = m >> filter(_._merge == "left_only")>>select(_.ntd_id_x,
                                                        _.agency_name,
                                                       _.doing_business_as, 
                                                        _.reporter_acronym,
                                                        _.organization_type_x,
                                                       _.reporter_type,
                                                       _.reported_by_ntd_id,
                                                       _.city)

In [51]:
nomatch2= m >> filter(_._merge == "right_only")>>select(_.name,
                                                        _.organization_type_y,
                                                        _.roles,
                                                        _.ntd_id_y,
                                                        _.itp_id, 
                                                        _.alias,
                                                        # _.opm_id_drmt,
                                                        # _.mobility_services_operated,
                                                        # _.mpo_rtpa
                                                       )

In [52]:
nomatch1>>filter(_.ntd_id_x==90260)

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city
116,90260,City of Compton,Compton Renaissance Transit Service,CRT,"City, County or Local Government Unit or Depar...",Reduced Reporter,NaN,Compton


In [53]:
nomatch2>>filter(_.ntd_id_y=='90260')

,name,organization_type_y,roles,ntd_id_y,itp_id,alias
916,City of Compton,City/Town,[],90260,77.0,[]


In [54]:
def get_num(x):
    try:
        return int(x)
    except Exception:
        try:
            return float(x)
        except Exception:
            return x  

In [55]:
nomatch1['ntd_id_x'] = nomatch1['ntd_id_x'].apply(get_num)
nomatch2['ntd_id_y'] = nomatch2['ntd_id_y'].apply(get_num)

In [56]:
nomatch1.sample()

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city
215,9R02-91093,City of Eureka,Eureka Transit Service,ETS,"City, County or Local Government Unit or Depar...",Rural Reporter,9R02,Eureka


In [57]:
nomatch_merge = pd.merge(nomatch1, nomatch2, left_on="doing_business_as", right_on="name", how="outer", indicator=True)

In [58]:
to_add1 = nomatch_merge>>filter(_._merge=='both')

In [59]:
to_add1

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntd_id_y,itp_id,alias,_merge


In [60]:
nomatch_merge2 = pd.merge(nomatch1, nomatch2, left_on="ntd_id_x", right_on="ntd_id_y", how="outer", indicator=True)

In [61]:
# no matches
to_add2 = nomatch_merge2>>filter(_._merge=='both')
to_add2

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,name,organization_type_y,roles,ntd_id_y,itp_id,alias,_merge
0,90007,City of Modesto,Modesto Area Express,MAX,"City, County or Local Government Unit or Depar...",Full Reporter,NaN,Modesto,City of Modesto,City/Town,[],90007,203.0,[MAX],both
1,90016,Golden Gate Bridge Highway and Transportation ...,NaN,GGBHTD,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,San Francisco,"Golden Gate Bridge, Highway and Transportation...",Independent Agency,[],90016,127.0,[],both
2,90022,City of Norwalk,Norwalk Transit System,NTS,"City, County or Local Government Unit or Depar...",Full Reporter,NaN,Norwalk,City of Norwalk,City/Town,[],90022,228.0,[],both
3,90024,City of La Mirada,La Mirada Transit,LMT,"City, County or Local Government Unit or Depar...",Full Reporter,NaN,La Mirada,City of La Mirada,City/Town,[],90024,151.0,[],both
4,90029,Omnitrans,NaN,NaN,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,San Bernardino,OmniTrans,Independent Agency,[],90029,232.0,[],both
5,90089,County of Sonoma,Sonoma County Transit,SCT,"City, County or Local Government Unit or Depar...",Full Reporter,NaN,Santa Rosa,Sonoma County,County,[],90089,314.0,[Healdsburg Shuttle],both
6,90091,City of Visalia,Visalia Transit,VT,"City, County or Local Government Unit or Depar...",Full Reporter,NaN,Visalia,City of Visalia,City/Town,[],90091,361.0,[],both
7,90092,City of Fairfield California,Fairfield and Suisun Transit,FAST,"City, County or Local Government Unit or Depar...",Full Reporter,NaN,Fairfield,City of Fairfield,City/Town,[],90092,110.0,[FAST],both
8,90142,University of California Davis,Unitrans,NaN,University,Full Reporter,NaN,Davis,"University of California, Davis",University - Public,[],90142,351.0,[],both
9,90144,Livermore Amador Valley Transit Authority,NaN,LAVTA,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Livermore,Livermore / Amador Valley Transit Authority,Independent Agency,[],90144,167.0,[LAVTA],both


In [62]:
len(to_add2)

38

In [69]:
## not working due to list in alias column from the airtable data
# nomatch_merge = pd.merge(nomatch1, nomatch2, left_on="reporter_acronym", right_on="alias", how="outer", indicator=True)

In [70]:
# to_add3 = nomatch_merge>>filter(_._merge=='both')
# to_add4 = (to_add3>>filter(_.alias.notnull()))
# to_add4

In [71]:
## Subsetting original merge

In [72]:
match = m>>filter(_._merge=='both')

In [73]:
match.sample()

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,alias,is_public_entity,ntd_id_y,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge
23,90035,Gold Coast Transit District,NaN,GCTD,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Oxnard,5f34debf19495b269d8d7ca8f7befdb9,recS7GnKTcQVX20HE,Gold Coast Transit District,Independent Agency,[],123.0,None,07 - Los Angeles,https://www.gctd.org/,Core,8566906637,Static OK,RT OK,True,Yes,[],True,90035,True,True,True,2023-09-06 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both


In [74]:
matches = pd.concat([match, to_add1], ignore_index=True, sort=False)

In [75]:
matches = pd.concat([matches, to_add2], ignore_index=True, sort=False)

/tmp/ipykernel_5535/1762955114.py:1: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  matches = pd.concat([matches, to_add2], ignore_index=True, sort=False)


In [77]:
## hashing out due to the add4 not working for data structure 
# matches = pd.concat([matches, to_add4], ignore_index=True, sort=False)

In [78]:
matches.sample()


,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,alias,is_public_entity,ntd_id_y,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge
37,A0013,Metropolitan Transportation Commission,NaN,MTC,"MPO, COG or Other Planning Agency",Group Plan Sponsor,NaN,San Francisco,4cbba16c0a3d48027736ae911cbd1489,recSOmBsATeoQylk1,Metropolitan Transportation Commission,MPO/RTPA,[Metropolitan Planning Organization],200.0,None,04 - Oakland,https://511.org/open-data/transit,None,1879187920,Static OK,RT Incomplete,False,Unknown,"[MTC, Bay Area Metro, Bay Area 511]",True,90094,False,False,True,2023-09-06 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both


In [79]:
len(matches)

223

* exporting to manipualte data 


In [80]:
#matches.to_csv('matches.csv')

In [81]:
#m.to_csv('ntd_itp_merge.csv')

## Merge Matches with 5310

* first fill in the missing information

In [82]:
matches>>filter(_.reporter_acronym.isnull())>>filter(_.alias.notnull())

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,alias,is_public_entity,ntd_id_y,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge
3,90008,City of Santa Monica,Big Blue Bus,NaN,"City, County or Local Government Unit or Depar...",Full Reporter,NaN,Santa Monica,fa75edea11e8c8012de543d85cc5a429,recJHFDLpGRMIFgnL,City of Santa Monica,City/Town,[],300.0,None,07 - Los Angeles,https://www.bigbluebus.com/,Core,1880607700,Static OK,RT OK,True,No,[],True,90008,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both
4,90009,San Mateo County Transit District,SamTrans,NaN,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,San Carlos,949e389a82b16005642e140407f2fff2,recw3mRsmKDTNnVlT,San Mateo County Transit District,Independent Agency,[],290.0,The San Mateo County Transit District is the a...,04 - Oakland,https://www.smctd.com/,Core,9261378702,Static OK,RT OK,True,Yes,[],True,90009,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both
8,90014,Alameda-Contra Costa Transit District,AC Transit,NaN,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Oakland,36d62623db3c7686370bc1b14475560e,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,Independent Agency,[],4.0,None,04 - Oakland,https://www.actransit.org/,Core,1880690867,Static OK,RT OK,True,Yes,[AC Transit],True,90014,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both
10,90017,City of Santa Rosa,NaN,NaN,"City, County or Local Government Unit or Depar...",Full Reporter,NaN,Santa Rosa,ec65a47b3c2636e16327da33fc22ca8d,recsNvWvLQwGsoxlE,City of Santa Rosa,City/Town,[],301.0,None,04 - Oakland,https://www.srcity.org/1036/Transit-and-CityBus,Core,1880784476,Static OK,RT OK,True,No,[],True,90017,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both
11,90019,Sacramento Regional Transit District,Sacramento RT,NaN,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Sacramento,6b21163bb0b51876670f03a46435902c,recX9lccSE1jmjsmG,Sacramento Regional Transit District,Independent Agency,[],273.0,None,03 - Marysville,http://www.sacrt.com/,Core,1880797919,Static OK,RT OK,True,No,"[SacRT, Sacramento RT]",True,90019,True,True,True,2023-09-16 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,9R02-91041,City of Dixon,Readi-Ride,NaN,"City, County or Local Government Unit or Depar...",Rural Reporter,9R02,Dixon,NaN,NaN,City of Dixon,City/Town,[],94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,9R02-91041,NaN,NaN,NaN,NaT,NaT,both
217,9R02-91082,Mariposa County Transit,Mari-Go,NaN,"City, County or Local Government Unit or Depar...",Rural Reporter,9R02,Mariposa,NaN,NaN,Mariposa County,County,[],195.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,9R02-91082,NaN,NaN,NaN,NaT,NaT,both
220,9R02-91095,Nevada County Transit Services,NaN,NaN,"City, County or Local Government Unit or Depar...",Rural Reporter,9R02,Nevada City,NaN,NaN,Nevada County,County,[],221.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Gold Country Stage],NaN,9R02-91095,NaN,NaN,NaN,NaT,NaT,both
221,9R02-99426,Wasco City of,NaN,NaN,"City, County or Local Government Unit or Depar...",Rural Reporter,9R02,Wasco,NaN,NaN,City of Wasco,City/Town,[],363.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,9R02-99426,NaN,NaN,NaN,NaT,NaT,both


In [83]:
matches.reporter_acronym.fillna(matches['alias'], inplace=True)

In [84]:
compare_col = np.where(matches["reporter_acronym"] == matches["alias"], True, False)
matches["compare_acronym"] = compare_col
matches.compare_acronym.value_counts()

False    126
True      97
Name: compare_acronym, dtype: int64

In [85]:
matches>>filter(_.compare_acronym==False)>>filter(_.reporter_acronym.notnull())>>filter(_.alias.notnull())>>select(_.agency_name, _.name, _.reporter_acronym, _.alias)

,agency_name,name,reporter_acronym,alias
0,San Francisco Bay Area Rapid Transit District,San Francisco Bay Area Rapid Transit District,BART,[]
1,Golden Empire Transit District,Golden Empire Transit District,GET,[]
2,Santa Cruz Metropolitan Transit District,Santa Cruz Metropolitan Transit District,SCMTD,[SCMTD]
5,City of Torrance,City of Torrance,TTS,[]
6,San Joaquin Regional Transit District,San Joaquin Regional Transit District,RTD,[SJRTD]
...,...,...,...,...
214,County of Siskiyou,Siskiyou County,STAGE,[STAGE]
215,Tuolumne County Transit,Tuolumne County Transit Agency,TCT,[]
216,Kern Regional Transit,Kern County,KT,[]
218,Glenn Transit Service,Glenn County,GTS,[]


* Only one not marching is San Diego Metropolitan Transit System, but MTS should be the right name
* Others are close

In [86]:
matches = matches.drop(columns={'alias'})

In [87]:
matches>>filter(_.organization_type_x.isnull())>>filter(_.organization_type_y.notnull())

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,is_public_entity,ntd_id_y,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge,compare_acronym
136,9R02,California Department of Transportation,NaN,Caltrans,NaN,State Reporter,NaN,Sacramento,92d05f4a5c01b51e8dbd7e4ff144631c,rec3DMyvpt33QLIwM,California Department of Transportation,Independent Agency,[],NaN,None,03 - Marysville,https://dot.ca.gov/,None,None,Static OK,RT Incomplete,False,Unknown,True,9R02,False,False,True,2023-05-24 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False


In [88]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 32 columns):
 #   Column                                  Non-Null Count  Dtype              
---  ------                                  --------------  -----              
 0   ntd_id_x                                223 non-null    object             
 1   agency_name                             223 non-null    object             
 2   doing_business_as                       73 non-null     object             
 3   reporter_acronym                        223 non-null    object             
 4   organization_type_x                     222 non-null    object             
 5   reporter_type                           223 non-null    object             
 6   reported_by_ntd_id                      60 non-null     object             
 7   city                                    223 non-null    object             
 8   key                                     185 non-null    object             
 9  

In [89]:
matches>>filter(_.roles.notnull())

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,is_public_entity,ntd_id_y,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge,compare_acronym
0,90003,San Francisco Bay Area Rapid Transit District,NaN,BART,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Oakland,0a00b255babc7b9b84442c0f8382cd7b,recoQLeNRISCKF8I0,San Francisco Bay Area Rapid Transit District,Independent Agency,[],279.0,None,04 - Oakland,https://www.bart.gov/,Core,1880583865,Static OK,RT OK,True,Yes,True,90003,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False
1,90004,Golden Empire Transit District,NaN,GET,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Bakersfield,49da62a5bf0a17437e9e5be0e44ab414,recIh3vq8jwuuJlvL,Golden Empire Transit District,Independent Agency,[],126.0,The Golden Empire Transit District (GET) was f...,06 - Fresno,https://www.getbus.org/,Core,1880627034,Static OK,RT OK,True,No,True,90004,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False
2,90006,Santa Cruz Metropolitan Transit District,NaN,SCMTD,Independent Public Agency or Authority of Tran...,Full Reporter,NaN,Santa Cruz,321cdc4c1f5d51c8848992b2fcc37729,recDFXyxaWQpiY4mw,Santa Cruz Metropolitan Transit District,Independent Agency,[],296.0,None,05 - San Luis Obispo,https://www.scmtd.com/en/,Core,1880583860,Static OK,RT OK,True,No,True,90006,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False
3,90008,City of Santa Monica,Big Blue Bus,[],"City, County or Local Government Unit or Depar...",Full Reporter,NaN,Santa Monica,fa75edea11e8c8012de543d85cc5a429,recJHFDLpGRMIFgnL,City of Santa Monica,City/Town,[],300.0,None,07 - Los Angeles,https://www.bigbluebus.com/,Core,1880607700,Static OK,RT OK,True,No,True,90008,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,True
4,90009,San Mateo County Transit District,SamTrans,[],Independent Public Agency or Authority of Tran...,Full Reporter,NaN,San Carlos,949e389a82b16005642e140407f2fff2,recw3mRsmKDTNnVlT,San Mateo County Transit District,Independent Agency,[],290.0,The San Mateo County Transit District is the a...,04 - Oakland,https://www.smctd.com/,Core,9261378702,Static OK,RT OK,True,Yes,True,90009,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,9R02-91088,Glenn Transit Service,NaN,GTS,Independent Public Agency or Authority of Tran...,Rural Reporter,9R02,Willows,NaN,NaN,Glenn County,County,[],122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9R02-91088,NaN,NaN,NaN,NaT,NaT,both,False
219,9R02-91093,City of Eureka,Eureka Transit Service,ETS,"City, County or Local Government Unit or Depar...",Rural Reporter,9R02,Eureka,NaN,NaN,City of Eureka,City/Town,[],108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9R02-91093,NaN,NaN,NaN,NaT,NaT,both,False
220,9R02-91095,Nevada County Transit Services,NaN,[Gold Country Stage],"City, County or Local Government Unit or Depar...",Rural Reporter,9R02,Nevada City,NaN,NaN,Nevada County,County,[],221.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9R02-91095,NaN,NaN,NaN,NaT,NaT,both,True
221,9R02-99426,Wasco City of,NaN,[],"City, County or Local Government Unit or Depar...",Rural Reporter,9R02,Wasco,NaN,NaN,City of Wasco,City/Town,[],363.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9R02-99426,NaN,NaN,NaN,NaT,NaT,both,True


In [90]:
# def recategorize(df):   
#     if (df.roles.str.contains('Metropolitan Planning Organization')):
#         return 'MPO/RTPA'
#     elif (df.roles.str.contains('Regional Transportation Planning Agency')) :
#         return 'MPO/RTPA'
#     elif (df.roles.str.contains('Regional Transportation Planning Agency,Metropolitan Planning Organization')):
#         return 'MPO/RTPA'
#     else: 
#         return NaN

In [91]:
#matches['organization_type_z'] = matches.apply(recategorize, axis = 1)

In [92]:
matches.organization_type_y.fillna(matches['roles'], inplace=True)

In [93]:
matches.sample()

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,is_public_entity,ntd_id_y,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge,compare_acronym
156,9R02-91053,Lake Transit Authority,NaN,LTA,Independent Public Agency or Authority of Tran...,Rural Reporter,9R02,Lower Lake,a973e445df4d0b878cd5d74b0ca4cc18,recPwXKbGLL4aIqXV,Lake Transit Authority,Independent Agency,[],159.0,None,01 - Eureka,https://laketransit.org/,Core,8564585553,Static OK,RT OK,True,No,True,9R02-91053,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False


In [94]:
matches>>filter(_.ntd_id_x.isnull())>>filter(_.ntd_id_y.notnull())

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,is_public_entity,ntd_id_y,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge,compare_acronym


In [95]:
matches = matches.drop(columns={'ntd_id_y'})
matches = matches.drop(columns={'reported_by_ntd_id'})


In [96]:
matches = matches.rename(columns={'ntd_id_x':'ntd_id'})

In [97]:
matches.sample(3)

,ntd_id,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,is_public_entity,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge,compare_acronym
118,90296,City of Claremont,Claremont Dial-a-Ride,CLDAR,"City, County or Local Government Unit or Depar...",Reduced Reporter,Claremont,45bfc6099611eae01e28c191f7f08063,rec740nNdh3RVBMkj,City of Claremont,City/Town,[],NaN,None,07 - Los Angeles,https://www.ci.claremont.ca.us/,None,1880627047,Static OK,RT Incomplete,False,Unknown,True,False,False,True,2023-05-25 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False
24,90044,City of Arcadia,Arcadia Transit,[Arcadia Transit],"City, County or Local Government Unit or Depar...",Reduced Reporter,Arcadia,aa85b1ea74f3c9d31dc883949ec16e2c,recjaoK08lDmhjQga,City of Arcadia,City/Town,[],17.0,None,07 - Los Angeles,https://www.arcadiaca.gov/shape/development_se...,Core,1880654216,Static OK,RT Incomplete,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,True
185,90007,City of Modesto,Modesto Area Express,MAX,"City, County or Local Government Unit or Depar...",Full Reporter,Modesto,NaN,NaN,City of Modesto,City/Town,[],203.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,both,False


* wondering if `opm_id_drmt` and `ali` match up

In [98]:
compare_col2 = np.where(matches["agency_name"] == matches["name"], True, False)
matches["compare_names"] = compare_col2
matches.compare_names.value_counts()

True     185
False     38
Name: compare_names, dtype: int64

In [99]:
matches>>filter(_.compare_names==False)>>select(_.agency_name, _.name)

,agency_name,name
185,City of Modesto,City of Modesto
186,Golden Gate Bridge Highway and Transportation ...,"Golden Gate Bridge, Highway and Transportation..."
187,City of Norwalk,City of Norwalk
188,City of La Mirada,City of La Mirada
189,Omnitrans,OmniTrans
190,County of Sonoma,Sonoma County
191,City of Visalia,City of Visalia
192,City of Fairfield California,City of Fairfield
193,University of California Davis,"University of California, Davis"
194,Livermore Amador Valley Transit Authority,Livermore / Amador Valley Transit Authority


## Merging 5310 Org names to CW

In [100]:
org_name.sample()

,organization_name
89,Livermore Amador Valley Transit Authority


In [101]:
matches.sample()

,ntd_id,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,is_public_entity,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge,compare_acronym,compare_names
30,90079,SunLine Transit Agency,NaN,[],Independent Public Agency or Authority of Tran...,Full Reporter,Thousand Palms,cedc3e7cf5339d552c4d5f39e722d119,recAsbHMwQWB7cri8,SunLine Transit Agency,Independent Agency,[],327.0,Transit agency around Palm Springs.,08 - San Bernardino,https://www.sunline.org/,Core,1880654228,Static OK,RT OK,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,True,True


In [102]:
m1 = (
    pd.merge(
        org_name,
        matches,
        left_on="organization_name",
        right_on="agency_name",
        how="outer",
        indicator="in_5310",
    )
)

In [103]:
len(m1>>filter(_.in_5310=='both'))

28

In [104]:
_5310_matches1 = m1>>filter(_.in_5310=='both')

In [105]:
_5310_matches1.sample(5)

,organization_name,ntd_id,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,is_public_entity,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge,compare_acronym,compare_names,in_5310
130,Colusa County Transit Agency,9R02-91112,Colusa County Transit Agency,NaN,[CCTA],"City, County or Local Government Unit or Depar...",Rural Reporter,Colusa,4047ad30ebd71bdf94b58de75a08b1c7,recwkt8N8YjUQTvuX,Colusa County Transit Agency,Independent Agency,[],74.0,Colusa County Transit consists of nine vehicle...,03 - Marysville,https://www.countyofcolusa.org/181/Transit,Core,1880654203,Static Incomplete,RT Incomplete,True,No,True,True,False,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,True,True,both
112,San Joaquin Regional Transit District,90012,San Joaquin Regional Transit District,NaN,RTD,Independent Public Agency or Authority of Tran...,Full Reporter,Stockton,ca5458ad1469e210cf8e60ec0a020310,recZgWVXkpix390of,San Joaquin Regional Transit District,Independent Agency,[],284.0,None,10 - Stockton,https://sanjoaquinrtd.com/,Core,1880583883,Static OK,RT Incomplete,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both
34,Morongo Basin Transit Authority,9R02-91090,Morongo Basin Transit Authority,NaN,MBTA,Independent Public Agency or Authority of Tran...,Rural Reporter,Joshua Tree,dabaf1c12b2aca15e506ce8fb4f7681e,recniaSbsXcRSLsWe,Morongo Basin Transit Authority,Independent Agency,[],212.0,None,08 - San Bernardino,https://mbtabus.com/,Other Public Transit,1880784470,Static OK,RT Incomplete,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both
121,City of Rio Vista,9R02-91014,City of Rio Vista,Rio Vista Delta Breeze,[],"City, County or Local Government Unit or Depar...",Rural Reporter,Rio Vista,c863a667cbbabeb5bdf0aab57aad9247,recaUmi7EJdPiG067,City of Rio Vista,City/Town,[],264.0,None,04 - Oakland,https://www.riovistacity.com/deltabreeze,Core,8564585554,Static OK,RT OK,True,Yes,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,True,True,both
54,Yolo County Transportation District,90090,Yolo County Transportation District,Yolobus,YCTD,Independent Public Agency or Authority of Tran...,Full Reporter,Woodland,36812a0f860dfbd4d46c0b8e92cdb0d2,recH2FdHvrL7nIpHA,Yolo County Transportation District,Independent Agency,[],372.0,None,03 - Marysville,http://www.yolobus.com/,Core,1880784499,Static OK,RT OK,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both


* merging on the other agency name to see if there are any more matches

In [106]:
_5310_matches2= (
    pd.merge(
        org_name,
        matches,
        left_on="organization_name",
        right_on="name",
        how="outer",
        indicator="in_5310",
    )
)

In [107]:
_5310_matches2.in_5310.value_counts()

right_only    194
left_only     120
both           29
Name: in_5310, dtype: int64

* same # of matches

In [108]:
#checking another name column

In [109]:
(
    pd.merge(
        org_name,
        matches,
        left_on="organization_name",
        right_on="doing_business_as",
        how="outer",
        indicator="in_5310",
    )
).in_5310.value_counts()

right_only    223
left_only     147
both            0
Name: in_5310, dtype: int64

#### Checking to see what is left for the agencies 

In [110]:
_5310_left = (m1>>filter(_.in_5310=='left_only'))

In [111]:
_5310_left = _5310_left>>select(_.organization_name)

In [112]:
# merging with unmatched.

In [113]:
unmatched = (m>>filter(_._merge != 'both'))

In [114]:
unmatched.sample(2)

,ntd_id_x,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,reported_by_ntd_id,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,alias,is_public_entity,ntd_id_y,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge
153,90298,County of Ventura,NaN,PWATD,"City, County or Local Government Unit or Depar...",Reduced Reporter,NaN,Ventura,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,left_only
580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b976f060e27bb8b3ed2c4d035118a664,recsqPNHcdEjtHIZA,City of Burlingame,City/Town,[],NaN,None,04 - Oakland,https://www.burlingame.org/,None,None,Static OK,RT Incomplete,False,Unknown,[],True,None,False,False,True,2023-05-25 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,right_only


In [115]:
(
    pd.merge(
        _5310_left,
        unmatched,
        left_on="organization_name",
        right_on="agency_name",
        how="outer",
        indicator="in_5310",
    )
).in_5310.value_counts()

right_only    789
left_only     121
both            0
Name: in_5310, dtype: int64

In [116]:
unmatch_merge2 = (
    pd.merge(
        _5310_left,
        unmatched,
        left_on="organization_name",
        right_on="name",
        how="outer",
        indicator="in_5310",
    )
)

In [117]:
unmatch_merge2.in_5310.value_counts()

right_only    756
left_only      88
both           33
Name: in_5310, dtype: int64

In [118]:
_5310_remaining2 = unmatch_merge2 >>filter(_.in_5310=='left_only')

In [119]:
unmatch_merge2 = unmatch_merge2 >>filter(_.in_5310=='both')

In [120]:
#for good measure
(
    pd.merge(
        _5310_left,
        unmatched,
        left_on="organization_name",
        right_on="doing_business_as",
        how="outer",
        indicator="in_5310",
    )
).in_5310.value_counts()

right_only    789
left_only     121
both            0
Name: in_5310, dtype: int64

#### Appending `_5310_left` to include these last matches

5310 matches from matches in ntd & itp
* _5310_matches1
* _5310_matches2

5310 matches from unmatches in ntd & itp
* unmatch_merge2

In [121]:
_5310_matches1 = _5310_matches1>>filter(_.in_5310==('both'))
_5310_matches2 = _5310_matches2>>filter(_.in_5310==('both'))

In [122]:
_5310_info = pd.concat([_5310_matches1, _5310_matches2], ignore_index=True, sort=False)

In [123]:
_5310_info = pd.concat([_5310_info, unmatch_merge2], ignore_index=True, sort=False)

/tmp/ipykernel_5535/2664687958.py:1: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  _5310_info = pd.concat([_5310_info, unmatch_merge2], ignore_index=True, sort=False)


In [129]:
## list error
# _5310_info = _5310_info.drop_duplicates()

In [127]:
_5310_info>>count(_.organization_name)>>filter(_.n>1)

,organization_name,n
6,City of Rio Vista,2
7,City of Roseville,2
8,City of Santa Rosa,2
9,City of Visalia,2
10,Colusa County Transit Agency,2
13,Eastern Sierra Transit Authority,2
14,El Dorado County Transit Authority,2
18,Humboldt Transit Authority,4
19,Imperial County Transportation Commission,2
23,Lake Transit Authority,2


* get rid of duplicates in downloaded version. these are single entries with double matches (on name and NTD_ids)

In [128]:
_5310_info

/opt/conda/lib/python3.9/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/opt/conda/lib/python3.9/site-packages/IPython/core/formatters.py:344: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,organization_name,ntd_id,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,is_public_entity,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge,compare_acronym,compare_names,in_5310,ntd_id_x,reported_by_ntd_id,alias,ntd_id_y
0,City of Roseville,90168,City of Roseville,Roseville Transit,[],"City, County or Local Government Unit or Depar...",Reduced Reporter,Roseville,573d134c3bb3b73b6f64c2586f25cd3a,recUdTq5QiUjJRiAe,City of Roseville,City/Town,[],271.0,None,03 - Marysville,https://www.roseville.ca.us/government/departm...,Core,9609215339,Static OK,RT Incomplete,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,True,True,both,NaN,NaN,NaN,NaN
1,Eastern Sierra Transit Authority,9R02-91062,Eastern Sierra Transit Authority,NaN,ESTA,Independent Public Agency or Authority of Tran...,Rural Reporter,Bishop,b2413ef48303343e08c7a1c81849ef9d,recRajLNJI206nnAr,Eastern Sierra Transit Authority,Independent Agency,[],99.0,None,09 - Bishop,https://www.estransit.com/,Core,1880531826,Static OK,RT OK,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both,NaN,NaN,NaN,NaN
2,Lake Transit Authority,9R02-91053,Lake Transit Authority,NaN,LTA,Independent Public Agency or Authority of Tran...,Rural Reporter,Lower Lake,a973e445df4d0b878cd5d74b0ca4cc18,recPwXKbGLL4aIqXV,Lake Transit Authority,Independent Agency,[],159.0,None,01 - Eureka,https://laketransit.org/,Core,8564585553,Static OK,RT OK,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both,NaN,NaN,NaN,NaN
3,Marin County Transit District,90234,Marin County Transit District,NaN,[],Independent Public Agency or Authority of Tran...,Full Reporter,San Rafael,7389f1baf873000e25cf84af403f6875,recNOb7pqBRlQVG5e,Marin County Transit District,Independent Agency,[],194.0,None,04 - Oakland,https://marintransit.org/,Core,1880690866,Static OK,RT OK,True,Yes,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,True,True,both,NaN,NaN,NaN,NaN
4,Morongo Basin Transit Authority,9R02-91090,Morongo Basin Transit Authority,NaN,MBTA,Independent Public Agency or Authority of Tran...,Rural Reporter,Joshua Tree,dabaf1c12b2aca15e506ce8fb4f7681e,recniaSbsXcRSLsWe,Morongo Basin Transit Authority,Independent Agency,[],212.0,None,08 - San Bernardino,https://mbtabus.com/,Other Public Transit,1880784470,Static OK,RT Incomplete,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Reach Out Morongo Basin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3d3f4331f907318ee0e4fb5e22fbed65,recSSO43Ib7ny2dXf,Reach Out Morongo Basin,Non-Profit Organization,[],454.0,None,08 - San Bernardino,http://reachoutmb.org/,#N/A,8567028105,Static OK,RT Incomplete,False,Unknown,False,False,False,True,2023-05-26 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,right_only,NaN,NaN,both,NaN,NaN,[],None
86,Porterville Sheltered Workshop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,696e17254c82f502983e1e048d24d761,rec0MHEjEQ4MLINEG,Porterville Sheltered Workshop,Non-Profit Organization,[],452.0,Independent living support for people with dis...,06 - Fresno,http://pswcares.org/,#N/A,8564585555,Static OK,RT Incomplete,False,Unknown,False,False,False,True,2023-04-29 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,right_only,NaN,NaN,both,NaN,NaN,[],None
87,Self-Help for the Elderly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9e515a57a898532533faeb5d1b0ff84f,recMDpMfW57uvb3CK,Self-Help for the E

In [130]:
#_5310_info.to_csv('5310_agencies_matches.csv')

* exporting to put information in for unmatched organizations

In [131]:
#_5310_remaining2.to_csv('5310_unmatched.csv')

In [136]:
_5310_info

/opt/conda/lib/python3.9/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/opt/conda/lib/python3.9/site-packages/IPython/core/formatters.py:344: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,organization_name,ntd_id,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,is_public_entity,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge,compare_acronym,compare_names,in_5310,ntd_id_x,reported_by_ntd_id,alias,ntd_id_y
0,City of Roseville,90168,City of Roseville,Roseville Transit,[],"City, County or Local Government Unit or Depar...",Reduced Reporter,Roseville,573d134c3bb3b73b6f64c2586f25cd3a,recUdTq5QiUjJRiAe,City of Roseville,City/Town,[],271.0,None,03 - Marysville,https://www.roseville.ca.us/government/departm...,Core,9609215339,Static OK,RT Incomplete,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,True,True,both,NaN,NaN,NaN,NaN
1,Eastern Sierra Transit Authority,9R02-91062,Eastern Sierra Transit Authority,NaN,ESTA,Independent Public Agency or Authority of Tran...,Rural Reporter,Bishop,b2413ef48303343e08c7a1c81849ef9d,recRajLNJI206nnAr,Eastern Sierra Transit Authority,Independent Agency,[],99.0,None,09 - Bishop,https://www.estransit.com/,Core,1880531826,Static OK,RT OK,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both,NaN,NaN,NaN,NaN
2,Lake Transit Authority,9R02-91053,Lake Transit Authority,NaN,LTA,Independent Public Agency or Authority of Tran...,Rural Reporter,Lower Lake,a973e445df4d0b878cd5d74b0ca4cc18,recPwXKbGLL4aIqXV,Lake Transit Authority,Independent Agency,[],159.0,None,01 - Eureka,https://laketransit.org/,Core,8564585553,Static OK,RT OK,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both,NaN,NaN,NaN,NaN
3,Marin County Transit District,90234,Marin County Transit District,NaN,[],Independent Public Agency or Authority of Tran...,Full Reporter,San Rafael,7389f1baf873000e25cf84af403f6875,recNOb7pqBRlQVG5e,Marin County Transit District,Independent Agency,[],194.0,None,04 - Oakland,https://marintransit.org/,Core,1880690866,Static OK,RT OK,True,Yes,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,True,True,both,NaN,NaN,NaN,NaN
4,Morongo Basin Transit Authority,9R02-91090,Morongo Basin Transit Authority,NaN,MBTA,Independent Public Agency or Authority of Tran...,Rural Reporter,Joshua Tree,dabaf1c12b2aca15e506ce8fb4f7681e,recniaSbsXcRSLsWe,Morongo Basin Transit Authority,Independent Agency,[],212.0,None,08 - San Bernardino,https://mbtabus.com/,Other Public Transit,1880784470,Static OK,RT Incomplete,True,No,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Reach Out Morongo Basin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3d3f4331f907318ee0e4fb5e22fbed65,recSSO43Ib7ny2dXf,Reach Out Morongo Basin,Non-Profit Organization,[],454.0,None,08 - San Bernardino,http://reachoutmb.org/,#N/A,8567028105,Static OK,RT Incomplete,False,Unknown,False,False,False,True,2023-05-26 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,right_only,NaN,NaN,both,NaN,NaN,[],None
86,Porterville Sheltered Workshop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,696e17254c82f502983e1e048d24d761,rec0MHEjEQ4MLINEG,Porterville Sheltered Workshop,Non-Profit Organization,[],452.0,Independent living support for people with dis...,06 - Fresno,http://pswcares.org/,#N/A,8564585555,Static OK,RT Incomplete,False,Unknown,False,False,False,True,2023-04-29 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,right_only,NaN,NaN,both,NaN,NaN,[],None
87,Self-Help for the Elderly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9e515a57a898532533faeb5d1b0ff84f,recMDpMfW57uvb3CK,Self-Help for the E

In [137]:
len(_5310_info>>count(_.organization_name)>>arrange(-_.n)>>filter(_.n>1))

27

In [140]:
## list error
# _5310_info = _5310_info.drop_duplicates()

In [141]:
#still have duplicates
len(_5310_info>>count(_.organization_name)>>arrange(-_.n)>>filter(_.n>1))

27

In [142]:
_5310_info>>filter(_.organization_name=='Humboldt Transit Authority')

,organization_name,ntd_id,agency_name,doing_business_as,reporter_acronym,organization_type_x,reporter_type,city,key,source_record_id,name,organization_type_y,roles,itp_id,details,caltrans_district,website,reporting_category,hubspot_company_record_id,gtfs_static_status,gtfs_realtime_status,_deprecated__assessment_status,manual_check__contact_on_website,is_public_entity,public_currently_operating,public_currently_operating_fixed_route,_is_current,_valid_from,_valid_to,_merge,compare_acronym,compare_names,in_5310,ntd_id_x,reported_by_ntd_id,alias,ntd_id_y
12,Humboldt Transit Authority,9R02-91036,Humboldt Transit Authority,NaN,HTA,Independent Public Agency or Authority of Tran...,Rural Reporter,Eureka,1e86981bcdda2637d734e600adf46bcc,recaa3naoNR4a5RsJ,Humboldt Transit Authority,Independent Agency,[],135.0,None,01 - Eureka,https://hta.org/,Core,1880531803,Static OK,RT OK,True,Yes,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both,NaN,NaN,NaN,NaN
13,Humboldt Transit Authority,A0009,Humboldt Transit Authority,NaN,HTA,Independent Public Agency or Authority of Tran...,Group Plan Sponsor,Eureka,1e86981bcdda2637d734e600adf46bcc,recaa3naoNR4a5RsJ,Humboldt Transit Authority,Independent Agency,[],135.0,None,01 - Eureka,https://hta.org/,Core,1880531803,Static OK,RT OK,True,Yes,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both,NaN,NaN,NaN,NaN
42,Humboldt Transit Authority,9R02-91036,Humboldt Transit Authority,NaN,HTA,Independent Public Agency or Authority of Tran...,Rural Reporter,Eureka,1e86981bcdda2637d734e600adf46bcc,recaa3naoNR4a5RsJ,Humboldt Transit Authority,Independent Agency,[],135.0,None,01 - Eureka,https://hta.org/,Core,1880531803,Static OK,RT OK,True,Yes,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both,NaN,NaN,NaN,NaN
43,Humboldt Transit Authority,A0009,Humboldt Transit Authority,NaN,HTA,Independent Public Agency or Authority of Tran...,Group Plan Sponsor,Eureka,1e86981bcdda2637d734e600adf46bcc,recaa3naoNR4a5RsJ,Humboldt Transit Authority,Independent Agency,[],135.0,None,01 - Eureka,https://hta.org/,Core,1880531803,Static OK,RT OK,True,Yes,True,True,True,True,2023-05-23 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00,both,False,True,both,NaN,NaN,NaN,NaN
